# 🧪 Küçük Dil Modeli (SLM) için komut istemi

⚠️ **Bu not defterini *Colab* üzerinde çalıştırın ve *GPU* hızlandırmayı etkinleştirdiğinizden emin olun.**

## 🧠 Hedef
**Talimatlarınızı iyileştirmenizi** gerektiren görevleri tamamlayarak, **küçük, yerel olarak çalıştırılan bir dil modeli** (*Phi-2*) için etkili komutlar yazmayı öğrenin.

---

## Φ Phi-2?

[Phi-2](https://www.microsoft.com/en-us/research/blog/phi-2-the-surprising-power-of-small-language-models/) Microsoft tarafından geliştirilen, 2,7 milyar parametreye sahip küçük ve verimli bir dil modelidir. Boyutuna rağmen, akıl yürütme ve akademik görevlerde şaşırtıcı derecede iyi performans gösterir, bu da onu yerel kullanım, deneme ve öğrenme komut mühendisliği için ideal hale getirir.

**Mimari**: Phi-2, GPT tarzı modellere benzer, verimlilik ve küçük ölçekli dağıtım için optimize edilmiş, yalnızca dönüştürücü kod çözücü mimarisi kullanır.

**Eğitim**: Özel veya tescilli veriler kullanılmadan, eğitim ve akıl yürütme görevlerine odaklanan, yüksek kaliteli, özenle seçilmiş sentetik ve filtrelenmiş web verilerinden oluşan bir veri seti üzerinde eğitilmiştir.

Phi-2 (ayrıca eski ve yeni Phi-x modelleri) Hugging Face'ten edinilebilir.

Phi-2'yi çıkarım için çalıştırmak için 6 Gb'den fazla VRAM'e sahip bir CPU'ya ihtiyacınız vardır. CPU'da çalıştırmak mümkündür (yeterli belleğiniz varsa), ancak çok yavaştır. Bu nedenle bu zorluğu Colab'da çalıştırıyoruz.

---

## 🧰 Kurulum Talimatları: Phi-2'yi `pipeline` ile çalıştırma

Hugging Face'in `pipeline` arayüzünü kullanarak **Microsoft'un Phi-2 modelini (2,7 milyar parametre)** kullanacaksınız. Bu, tokenizasyonu manuel olarak işlemekten daha kolay ve temizdir.

### Adım 1: Gerekli Paketleri Yükleyin

In [ ]:
# Yerel olarak çalıştırıyorsanız yorum satırını kaldırın.
# !pip install transformers accelerate torch

### Adım 2: Phi-2'yi `pipeline` ile yükleyin

In [ ]:
from transformers import pipeline

generator = pipeline(
    "text-generation",
    model="microsoft/phi-2",
    torch_dtype="auto",
    device_map="auto"
)

### Adım 3: Yanıt Oluşturma

In [ ]:
prompt = "What causes the seasons?"
response = generator(prompt, max_new_tokens=100)[0]["generated_text"]

In [ ]:
# Metni güzel bir şekilde biçimlendirdiği için yanıtı yazdırmak yerine Markdown ile görüntüleyelim.
from IPython.display import Markdown
Markdown(response)

Yanıtlarımızın ne kadar hızlı tekrarlandığını görüyor musunuz? Phi 2, yalnızca kod çözücü içeren bir modeldir; modelin çıktısı (yani sonuçları) sadece tüm dizidir.

Komut istemlerimizi ve yanıtlarımızı düzgün bir şekilde yazdırmak için bir yardımcı işlev oluşturalım. 👉 Aşağıdaki hücreyi çalıştırın:

In [ ]:
def show_results(prompt, response):
    """Display the prompt and response in a formatted way.
    Excludes the prompt in the response to avoid repetition."""
    display(Markdown(
        "### Prompt:\n"
        + prompt
        + "\n### Response:\n"
        + response[len(prompt):]
        + "\n\n---"
    ))

---

## 🧩 Komut İsteği Görevleriniz

Her görev için aşağıdaki talimatları izleyin:

👉 İlk komut isteğini yazın.

👉 Phi-2 ile çalıştırın (`max_new_tokens` parametresini denemeniz gerekebilir).

👉 Komut isteğini iyileştirin.

👉 Sonuçları karşılaştırın.

👉 Ancak o zaman çözüme bakın.

---

### 📝 Görev 1: Temel Soru Yanıtlama

In [ ]:
# Adım 1: İlk komut istemini deneyin
prompt = "What causes the seasons?"
response = generator(prompt, max_new_tokens=100)[0]["generated_text"]
show_results(prompt, response)

Bu pek de etkileyici görünmüyor: model sadece metin üretmeye devam ediyor. Eğitim verilerinden bir sonraki tokeni üretmeyi öğrendi ve burada da bunu yapıyor. *Sıra sonu* tokeni üretmediği sürece devam edecek.

Model, GPT-3.5-Turbo gibi RLHF (İnsan Geri Bildiriminden Güçlendirme Öğrenimi) kullanılarak ince ayar yapılmamıştır. Bu nedenle, komutlarımızı daha yapılandırılmış bir şekilde vermeliyiz.

👉 Başka bir şey deneyelim:

In [ ]:
# Adım 2: Promptunu iyileştir
prompt2 = "Explain in simple terms: What causes the seasons?"
response2 = generator(prompt2, max_new_tokens=100)[0]["generated_text"]
show_results(prompt, response)

Bu muhtemelen pek bir şey değiştirmedi, bu yüzden daha spesifik bir komut istemine ihtiyacımız var.

Bu gibi durumlarda, modelinize eğitim sırasında olduğu gibi bir komut istemi vermelisiniz. 

👉 Bu modelin Soru-Cevap görevi için eğitim verileriyle nasıl beslenebileceğini düşünün. Ona bir soru ve bir cevap verilmiş olurdu. Bunu bilerek, yeni bir komut istemi deneyin.

In [ ]:
# SENİN KODUN BURAYA

Nasıl çalışması gerektiğini tahmin etmekten bıktınız mı? 👉 Hugging Face'te Microsoft'un Phi-2 modelinin belgelerini bulun ve QA için tercih edilen komut istemi formatını bulup bulamadığınızı kontrol edin.

<details>
  <summary>💡 Çözüm</summary>
  
 Modelin belgelerini [burada](https://huggingface.co/microsoft/phi-2) bulabilirsiniz.
  
  Görünüşe göre komut istemini şu şekilde biçimlendirmek gerekiyor:

```text
  Talimat: Sorunuzu buraya yazın.
  Çıktı:
  ```

  Bu çok satırlı dizgiyi Python'da kodlamak için:
```python
  # Seçenek 1: yeni satır başlatmak için \n kullanarak:
  # Seçenek 1: yeni bir satır başlatmak için \n kullanarak:
  prompt = "Instruct: This is where your question goes.\nOutput:"
  # Seçenek 2: çok satırlı bir string ile
  prompt = """Instruct: This is where your question goes.
  Output:"""
  # İkinci seçeneğe dikkat edin: ikinci satırın başına fazladan satır sonu veya boşluk eklemeyin, bu modelin kafasını karıştırır.
  ```

 Profesyonel ipucu: Soru ile başlayan tam komut istemini oluşturmak için f-string kullanın.
</details>

---
### 📝 Görev 2: Özetleme

Bazı metinleri özetlemeye çalışalım. Bu, Wikipedia'nın transformatörler sayfasında yer alan bir alıntıdır:

In [ ]:
text = """
Transformers is a media franchise produced by Japanese toy company Takara Tomy and American toy company Hasbro. It primarily follows the heroic Autobots and the villainous Decepticons, two alien robot factions at war that can transform into other forms, such as vehicles and animals. The franchise encompasses toys, animation, comic books, video games and films. As of 2011, it generated more than ¥2 trillion ($25 billion) in revenue,[1] making it one of the highest-grossing media franchises of all time.

The franchise began in 1984 with the Transformers toy line, comprising transforming mecha toys from Takara's Diaclone and Micro Change toylines rebranded for Western markets.[2] The term "Generation 1" (G1) covers both the animated television series The Transformers and the comic book series of the same name, which are further divided into Japanese, British and Canadian spin-offs. Sequels followed, such as the Generation 2 comic book and Beast Wars TV series, which became its own mini-universe. Generation 1 characters have been rebooted multiple times in the 21st century in comics from Dreamwave Productions (starting 2001), IDW Publishing (starting in 2005 and again in 2019), and Skybound Entertainment (beginning in 2023). There have been other incarnations of the story based on different toy lines during and after the 20th century. The first was the Robots in Disguise series, followed by three shows (Armada, Energon, and Cybertron) that constitute a single universe called the "Unicron Trilogy".
"""
Markdown(text)

👉 Modele kısa bir özet oluşturmasını isteyin. Bunun, `max_new_tokens` ayarınız nedeniyle kısa olmadığından emin olun.

In [ ]:
prompt = f"Input: {text}\nSummary: "
response = generator(prompt, max_new_tokens=200)[0]["generated_text"]
show_results(prompt, response)

<details>
  <summary>💡 Nereden başlayacağınızı bilmiyor musunuz?</summary>
  
  Şuradan başlayabilirsiniz:

```text
  Özetleyin: Özetlenecek metin burada yer alır.
  ```

  Modelin daha kısa bir özet oluşturmasını sağlayın.

</details>

<details>
  <summary>💡 Çözüm</summary>
  
  Kısa bir özet elde etmek için, bu iyi sonuçlar veriyor gibi görünüyor:

```text
  Bunu tek bir cümleyle özetleyin: İşte metniniz.
  ```

  Ancak model muhtemelen bu şekilde eğitilmemiştir.

  Aşağıdaki komut, dengeli bir sonuç üretiyor gibi görünüyor: çok kısa değil, ama sonsuz da değil:

```text
  Input: İşte metniniz.
  Summary:
  ```

  Ya da bunun kadar kısa bir şey:

  ```text
  İşte metniniz. TLDR:
  ```

  Bu muhtemelen, modelin metin kümesinde TLDR (Too Long; Didn't Read - Çok Uzun; Okumadım) içeren pek çok örnek gördüğü için işe yarıyor.
  
</details>

---
### 📝 Görev 3: Adım Adım Akıl Yürütme

Modele soruları çözmesini de isteyebiliriz.

👉 Aşağıdaki komutu deneyin:

In [ ]:
prompt = "If Alice has 3 apples and buys 2 more, then gives one away, how many does she have left?"
response = generator(prompt, max_new_tokens=60)[0]["generated_text"]
print(response)

Beklediğiniz bu muydu?

Hayır, model kod üretiyor gibi görünüyor. İstediğimiz şey bu değil (en azından burada değil, takipte kalın).

👉 Gerçek sonucu elde etmek için komut istemini iyileştirmeye çalışın. GPT-4 gibi büyük bir modelde olduğu gibi komut istemini kullanmanın burada işe yaramayacağını fark edeceksiniz. Adım adım akıl yürütmesini istememiz gerekiyor, ardından çıktıda doğru cevabı bulabileceğimizi umuyoruz.

In [ ]:
# SENİN KODUN BURAYA

<details>
  <summary>💡 Çözüm</summary>
  
  Kısa bir özet için, bu iyi sonuçlar veriyor gibi görünüyor:

```text
  Alice'in 3 elması varsa ve 2 tane daha satın alıp birini başkasına verirse, elinde kaç tane kalır? Adım adım çözün.
  ```

</details>

Bu çok uzun oldu. Modeli yönlendirmek için başka yollar düşünebilir misin?

👉 Belgelere tekrar bir göz at.

<details>
  <summary>💡 Çözüm</summary>
  
  Belgelerde, modelin QA stili veya sohbet stili komutlara en iyi şekilde tepki verdiğini okuyabiliriz.

  Bu şekilde komut vermeye çalışın. Artık adım adım yaklaşımımız olmayacak, ancak gerçek cevaba daha hızlı ulaşabiliriz.
</details>

👉 Sohbet stilini kullanmayı deneyin:

In [ ]:
# SENİN KODUN BURAYA

👉 Ve QA stili:

In [ ]:
# SENİN KODUN BURAYA

<details>
  <summary>💡 Çözüm</summary>
  
  Sohbet stili:
```text
  Öğretmen: İşte soru.
  Öğrenci:
  ```

Soru-cevap stili:
```text
  Instruct: İşte soru.
  Output:
  ```
</details>

---
### 📝 Görev 4: Classification

Bazı film eleştirilerini derecelendirmeyi deneyelim.

👉 [Kaggle'dan IMDB Veri Setini indirin](https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews?select=IMDB+Dataset.csv) ve Colab'a yükleyin. Ardından aşağıdaki hücreyi çalıştırarak verileri yükleyin.

In [ ]:
import pandas as pd
reviews = pd.read_csv("./IMDB Dataset.csv", sep=",")['review']

In [ ]:
review = reviews[0]  # Bu endeksi kullanarak farklı incelemelerle test edin
prompt = f"Classify the sentiment of this review as Positive, Neutral, or Negative: '{review}'"
response = generator(prompt, max_new_tokens=40)[0]["generated_text"]
show_results(prompt, response)

Pek iyi değil, değil mi? Unutmayın: bu bir üretken modeldir, yani sonraki tokenleri üretir. Komut istemimizde biraz daha akıllı davranmamız gerekecek.

👉 Önce komut istemini kendiniz iyileştirmeye çalışın. Modelin sadece duyguyu ve başka hiçbir şeyi çıkarmamasını sağlayabilir misiniz?

In [ ]:
# SENİN KODUN BURAYA

<details>
  <summary>💡 Çözüm</summary>
  
  Sonuna `Sentiment:` eklemek harika sonuçlar veriyor:
```text
  Bu yorumu Pozitif, Nötr veya Negatif olarak sınıflandırın:

  İşte yorum.

  Sentiment:
  ```

Muhtemelen model bu formattaki verileri gördüğü içindir.

</details>

---
### 📝 Görev 5: Kod oluşturma

Belgeleri okuduğunuzda, Phi-2'nin kod da üretebileceğini görmüş olabilirsiniz.

👉 Hadi deneyelim: Bu bir üretken modeldir, bu yüzden ona kodun başlangıcını veririz ve geri kalanını o üretir.

In [ ]:
code_start = '''
def get_weather(location, fahrenheit=False):
'''
response = generator(code_start, max_new_tokens=200)[0]["generated_text"]
print(response)

Verdiğimiz sınırlı bilgiye göre fena değil. Nasıl daha iyisini yapabiliriz? Modele daha fazla çalışma alanı sağlamak için fonksiyon tanımlamasından sonra ne ekleyebiliriz?

👉 Promptunuzu iyileştirmeye çalışın.

In [ ]:
# SENİN KODUN BURAYA

<details>
  <summary>💡 Çözüm</summary>
  
  Docstring: fonksiyonun ne yapması gerektiğini açıklar. Bu, model için harika bir talimat görevi görür.

  Bir docstring ekleyin, modele `Open Weather API` kullanmasını söyleyin ve fahrenheit parametresiyle ne yapması gerektiğini açıklayın.

</details>

👉 Kodu inceleyin. Size doğru görünüyor mu? [Open Weather API belgeleriyle](https://openweathermap.org/current) karşılaştırın.

<details>
  <summary>💡 Çözüm</summary>
  
  Kod muhtemelen sorunlu görünmüyor. Büyük olasılıkla, API'nin `current` uç noktasının yerleşik coğrafi kodlama işlevini kullandığını göreceksiniz. Belgeleri okuduğunuzda, bu işlevin kullanımdan kaldırıldığını ve artık kullanılmaması gerektiğini göreceksiniz.

Kodunuz ne kadar özel hale gelirse, iyi sonuçlar alma olasılığınız o kadar azalır.

</details>

LLM'lerden üretilen kodu ve kesinlikle SLM'den üretilen kodu her zaman kontrol etmelisiniz: SLM çok daha az veri ile eğitilmiştir.

👉 Kod üretimi için [Phi-2'nin sınırlamaları](https://huggingface.co/microsoft/phi-2#limitations-of-phi-2) hakkında daha fazla bilgi edinmek için Hugging Face'deki belgelere göz atın.

---

🏁 Tebrikler! Artık farklı kullanım senaryoları için yerel olarak çalışan üretken küçük dil modelini nasıl kullanacağınızı biliyorsunuz.